In [1]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
from biogeme.expressions import Beta
from biogeme.expressions import log
import math

In [2]:
df = pd.read_csv("data/us_estdata.csv")
df.head()

,Unnamed: 0,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,ST,ADJINC,PWGTP,...,EDU_ORIG,MED_ORIG,ENT_ORIG,FOD_ORIG,SRV_ORIG,PUB_ORIG,JOBS_EDU_NOHS_ORIG,JOBS_EDU_HS_ORIG,JOBS_EDU_NOBACH_ORIG,JOBS_EDU_BACH_ORIG
0,0,P,2018GQ0103142,2,1,902,1,34,1013097,27,...,38315.0,54857.0,4668.0,25243.0,13545.0,12088.0,44574.0,75277.0,96405.0,139711.0
1,1,P,2018HU1096416,9,1,10200,4,53,1013097,341,...,7051.0,9608.0,2445.0,7125.0,2552.0,3823.0,6449.0,14893.0,18922.0,14092.0
2,2,P,2018HU0306186,7,2,1400,3,48,1013097,43,...,2964.0,3722.0,179.0,3216.0,886.0,688.0,4016.0,6496.0,7050.0,4215.0
3,3,P,2018HU0641097,7,1,1200,3,48,1013097,24,...,3558.0,3012.0,97.0,2808.0,898.0,988.0,3882.0,7306.0,7491.0,4446.0
4,4,P,2018HU1246748,5,1,7300,3,12,1013097,75,...,18126.0,21139.0,2060.0,17590.0,6016.0,35219.0,15006.0,31118.0,38494.0,34392.0


In [3]:
df["MED"] = np.where(df["NAICS"] == "MED", 1, 0)
df["MFG"] = np.where(df["NAICS"] == "MFG", 1, 0)
df["RET"] = np.where(df["NAICS"] == "RET", 1, 0)
df["EDU"] = np.where(df["NAICS"] == "EDU", 1, 0)
df["ADM"] = np.where(df["NAICS"] == "ADM", 1, 0)
df["FOD"] = np.where(df["NAICS"] == "FOD", 1, 0)
df["PRF"] = np.where(df["NAICS"] == "PRF", 1, 0)
df["TRN"] = np.where(df["NAICS"] == "TRN", 1, 0)
df["SRV"] = np.where(df["NAICS"] == "SRV", 1, 0)
df["FIN"] = np.where(df["NAICS"] == "FIN", 1, 0)
df["WHL"] = np.where(df["NAICS"] == "WHL", 1, 0)
df["AGR"] = np.where(df["NAICS"] == "AGR", 1, 0)
df["PUB"] = np.where(df["NAICS"] == "PUB", 1, 0)
df["INF"] = np.where(df["NAICS"] == "INF", 1, 0)
df["ENT"] = np.where(df["NAICS"] == "ENT", 1, 0)
df["REL"] = np.where(df["NAICS"] == "REL", 1, 0)
df["UTL"] = np.where(df["NAICS"] == "UTL", 1, 0)
df["EXT"] = np.where(df["NAICS"] == "EXT", 1, 0)
df["MNG"] = np.where(df["NAICS"] == "MNG", 1, 0)
df["CON"] = np.where(df["NAICS"] == "CON", 1, 0)

In [4]:
# clean up the database (Biogeme Database can only have numerical values)
df = df.select_dtypes(['number'])
df = df.fillna(0)

In [5]:
df["CHOSEN"].value_counts()

102500     106
5500100     85
5310200     84
5500700     74
1700202     73
          ... 
4804619      7
2701404      7
5541001      6
2601703      5
3500806      4
Name: CHOSEN, Length: 2336, dtype: int64

In [6]:
# defining the chosen alterantive for each person explicitly (0 to 35, corresponding to staying and moving to one of the many PUMAs)
df['CHOSEN_PUMA'] = df['CHOSEN']
df['CHOSEN'] = 0
for i in range(1, 101): 
    var = 'ALT' + str(i) + '_PUMA'
    df['CHOSEN'] = np.where(df[var]==df['CHOSEN_PUMA'], i, df['CHOSEN'])
df["CHOSEN"] = np.where(df["STAY"] == 1, 0, df["CHOSEN"])

In [7]:
df["CHOSEN"].value_counts()

0    55261
1     8011
Name: CHOSEN, dtype: int64

In [8]:
df["IN_COLLEGE"] = np.where((df["SCHG"] == 15) | (df["SCHG"] == 16), 1, 0)
df["IN_COLLEGE"]

0        0
1        0
2        0
3        0
4        0
        ..
63267    0
63268    0
63269    0
63270    0
63271    0
Name: IN_COLLEGE, Length: 63272, dtype: int32

In [9]:
df["AGE_18_34"] = np.where(df["AGEP"] <= 34, 1, 0)
df["AGE_35_64"] = np.where((df["AGEP"] >= 35) & (df["AGEP"] <= 64), 1, 0)
df["AGE_OVER_65"] = np.where((df["AGEP"] >= 65), 1, 0)
df["FOREIGN"] = np.where(df["NATIVITY"] == 2, 1, 0)

In [10]:
df["AGE_18_22"] = np.where(df["AGEP"] <= 22, 1, 0)
df["AGE_23_29"] = np.where((df["AGEP"] >= 23) & (df["AGEP"] <= 29), 1, 0)
df["AGE_30_39"] = np.where((df["AGEP"] >= 30) & (df["AGEP"] <= 39), 1, 0)
df["AGE_40_49"] = np.where((df["AGEP"] >= 40) & (df["AGEP"] <= 49), 1, 0)
df["AGE_50_64"] = np.where((df["AGEP"] >= 50) & (df["AGEP"] <= 64), 1, 0)

In [11]:
df["EDU_LESS_HIGH"] = np.where(df["SCHL"] <= 15, 1, 0)
df["EDU_HIGHONLY"] = np.where((df["SCHL"] >= 16) & (df["SCHL"] <= 17), 1, 0)
df["EDU_SOMECOLLEGE"] = np.where((df["SCHL"] >= 18) & (df["SCHL"] <= 20), 1, 0)
df["EDU_COLLEGE"] = np.where(df["SCHL"] >= 21, 1, 0)
df["EDU_NOCOLLEGE"] = np.where(df["EDU_COLLEGE"] == 0, 1, 0)

In [12]:
df["WOMAN_CHILD"] = np.where((df["PAOC"] >= 1) & (df["PAOC"] <= 3), 1, 0)
df["UNEMPLOYED"] = np.where(df["ESR"] == 3, 1, 0)

In [13]:
df["MALE"] = np.where(df["SEX"] == 1, 1, 0)
df["FEMALE"] = np.where(df["SEX"] == 0, 1, 0)

In [14]:
df["MARRIED"] = np.where(df["MAR"] == 1, 1, 0)

In [15]:
df["child_old"] = np.where(df["child"] == df["REC_CHILD"], 0, df["child"])
df["child_old"].value_counts()

0    44611
1    18661
Name: child_old, dtype: int64

In [16]:
df["REC_NO_MAR"] = np.where((df["MARHD"] == 1) | (df["MARHW"] == 1), 1, 0)
df["REC_NO_MAR"].value_counts()

0    62389
1      883
Name: REC_NO_MAR, dtype: int64

In [17]:
df["MARHM_new"] = np.where(df["MARHM"] == 2, 0, df["MARHM"])
df["MARHM_new"].value_counts()

0.0    62261
1.0     1011
Name: MARHM_new, dtype: int64

In [18]:
df["MARRIED"].value_counts()

1    34051
0    29221
Name: MARRIED, dtype: int64

In [19]:
df["married_old"] = np.where((df["MARHM"] == df["MARRIED"]), 0, df["MARRIED"])
df["married_old"].value_counts()

1    33059
0    30213
Name: married_old, dtype: int64

In [20]:
df["MILITARY"] = np.where(df["MIL"] == 1, 1, 0)

In [21]:
# making the Biogeme Database that is used for the model estimation
database = db.Database('us_data', df)

In [22]:
# The following statement allows you to use the names of the
# variable as Python variable. (in the utility functions)
globals().update(database.variables)

In [23]:
# Destination Choice Parameters to be estimated
# Beta(name of the factor, initial value of the coefficient, lower bound, upper bound, whether or not the coefficinet should be fixed to initial value value)
c_destchoice_emp=Beta('c_destchoice_emp', 0, None, None, 0)
c_destchoice_dist=Beta('c_destchoice_dist', 0, None, None, 0)
c_destchoice_logdist=Beta('c_destchoice_logdist', 0, None, None, 0)
c_destchoice_time=Beta('c_destchoice_time', 0, None, None, 0)
c_destchoice_unemp = Beta("c_destchoice_unemp", 0, None, None, 0)
c_destchoice_hhinc = Beta("c_destchoice_hhinc", 0, None, None, 0)
c_destchoice_internal = Beta("c_destchoice_internal", 0, None, None, 0)
c_destchoice_urban = Beta("c_destchoice_urban", 0, None, None, 0)
c_destchoice_huprice = Beta("c_destchoice_huprice", 0, None, None, 0)
c_destchoice_hurent = Beta("c_destchoice_hurent", 0, None, None, 0)
c_destchoice_vacancy = Beta("c_destchoice_vacancy", 0, None, None, 0)
c_destchoice_college = Beta("c_destchoice_college", 0, None, None, 0)
c_destchoice_age_18_34 = Beta("c_destchoice_age_18_34", 0, None, None, 0)
c_destchoice_age_35_64 = Beta("c_destchoice_age_35_64", 0, None, None, 0)
c_destchoice_age_over_65 = Beta("c_destchoice_age_over_65", 0, None, None, 0)
c_destchoice_foreign = Beta("c_destchoice_foreign", 0, None, None, 0)
c_destchoice_unemp = Beta("c_destchoice_unemp", 0, None, None, 0)
c_destchoice_pctbach = Beta("c_destchoice_pctbach", 0, None, None, 0)
c_destchoice_pctnobach = Beta("c_destchoice_pctnobach", 0, None, None, 0)
c_destchoice_pctownind = Beta("c_destchoice_pctownind", 0, None, None, 0)
c_destchoice_entscore = Beta("c_destchoice_entscore", 0, None, None, 0)

In [24]:
c_move = Beta("c_move", 0, None, None, 0)

In [25]:
# Staying Choice Parameters to be Estimated
c_stay_married = Beta("c_stay_married", 0, None, None, 0)
c_stay_income = Beta("c_stay_income", 0, None, None, 0)
c_stay_origin_unemp_rate = Beta("c_stay_origin_unemp", 0, None, None, 0)
c_stay_age_18_22 = Beta("c_stay_age_18_22", 0, None, None, 0)
c_stay_age_23_29 = Beta("c_stay_age_23_29", 0, None, None, 0)
c_stay_age_30_39 = Beta("c_stay_age_30_39", 0, None, None, 0)
c_stay_age_50_64 = Beta("c_stay_age_50_64", 0, None, None, 0)
c_stay_age_65 = Beta("c_stay_age_65", 0, None, None, 0)
c_stay_edu_nohigh = Beta("c_stay_edu_nohigh", 0, None, None, 0)
c_stay_edu_somecollege = Beta("c_stay_edu_somecollege", 0, None, None, 0)
c_stay_edu_college = Beta("c_stay_edu_college", 0, None, None, 0)
c_stay_child = Beta("c_stay_child", 0, None, None, 0)
c_stay_unemployed = Beta("c_stay_unemployed", 0, None, None, 0)
c_stay = Beta("c_stay", 0, None, None, 0)
c_stay_hhinc_orig = Beta("c_stay_hhinc_orig", 0, None, None, 0)
c_stay_foreign = Beta("c_stay_foreign", 0, None, None, 0)
c_stay_rent = Beta("c_stay_rent", 0, None, None, 0)
c_stay_hurent = Beta("c_stay_hurent", 0, None, None, 0)
c_stay_dens = Beta("c_stay_dens", 0, None, None, 0)
c_stay_college = Beta("c_stay_college", 0, None, None, 0)

In [26]:
c_stay_rec_child = Beta("c_stay_rec_child", 0, None, None, 0)
c_stay_rec_mar = Beta("c_stay_rec_mar", 0, None, None, 0)
c_stay_rec_nomar = Beta("c_stay_rec_nomar", 0, None, None, 0)

In [27]:
# occupation-based coefficients

c_adm = Beta("c_adm", 0, None, None, 0)
c_agr = Beta("c_agr", 0, None, None, 0)
c_con = Beta("c_con", 0, None, None, 0)
c_edu = Beta("c_edu", 0, None, None, 0)
c_ent = Beta("c_ent", 0, None, None, 0)
c_ext = Beta("c_ext", 0, None, None, 0)
c_fin = Beta("c_fin", 0, None, None, 0)
c_fod = Beta("c_fod", 0, None, None, 0)
c_inf = Beta("c_inf", 0, None, None, 0)
c_med = Beta("c_med", 0, None, None, 0)
c_mfg = Beta("c_mfg", 0, None, None, 0)
c_mng = Beta("c_mng", 0, None, None, 0)
c_prf = Beta("c_prf", 0, None, None, 0)
c_pub = Beta("c_pub", 0, None, None, 0)
c_rel = Beta("c_rel", 0, None, None, 0)
c_ret = Beta("c_ret", 0, None, None, 0)
c_srv = Beta("c_srv", 0, None, None, 0)
c_trn = Beta("c_trn", 0, None, None, 0)
c_utl = Beta("c_utl", 0, None, None, 0)
c_whl = Beta("c_whl", 0, None, None, 0)
c_stay_mil = Beta("c_stay_mil", 0, None, None, 0)

In [28]:
# defining the staying utility function
V0 = c_stay_married * married_old + c_stay_income * PINCP + c_stay_age_18_22 * AGE_18_22 + c_stay_age_23_29 * AGE_23_29 + c_stay_age_30_39 * AGE_30_39 + c_stay_age_50_64 * AGE_50_64 + c_stay_age_65 * AGE_OVER_65 + c_stay_edu_college * EDU_COLLEGE + c_stay_child * child_old + c_stay + c_stay_hhinc_orig * HH_MED_INC_ORIG + c_stay_foreign * FOREIGN + c_stay_hurent * HH_MED_RENT_ORIG + c_stay_dens * DENS_ORIG + c_stay_college * IN_COLLEGE + c_stay_rec_mar * MARHM_new + c_stay_rec_nomar * REC_NO_MAR + c_stay_rec_child * REC_CHILD + c_stay_mil * MILITARY

# c_stay_married * MARRIED + c_stay_income * PINCP + c_stay_age_18_22 * AGE_18_22 + c_stay_age_23_29 * AGE_23_29 + c_stay_age_30_39 * AGE_30_39 + c_stay_age_50_64 * AGE_50_64 + c_stay_age_65 * AGE_OVER_65 + c_stay_edu_nohigh * EDU_LESS_HIGH + c_stay_edu_somecollege * EDU_SOMECOLLEGE + c_stay_edu_college * EDU_COLLEGE + c_stay_child * child + c_stay + c_stay_hhinc_orig * HH_MED_INC_ORIG + c_stay_foreign * FOREIGN + c_stay_hurent * HH_MED_RENT_ORIG + c_stay_dens * DENS_ORIG + c_stay_origin_unemp_rate * UNEMP_ORIG + c_stay_college * IN_COLLEGE

In [29]:
# defining the utility functions for each of the moving PUMA alternatives
# defined using the exec to parse a string to save space
# can also use a loop to print out the statements and then copy/paste them to run
# can also just write each one manually
for i in range(100):
    num = i + 1
    initialization = "V{0} = log(ALT{0}_POP) + c_destchoice_dist * ALT{0}_DIST + c_destchoice_logdist * log(ALT{0}_DIST + 1) + c_destchoice_urban / DENS_ORIG * (ALT{0}_DENS - DENS_ORIG) + c_destchoice_hurent * ALT{0}_HURENT + c_destchoice_college * IN_COLLEGE * ALT{0}_COLLEGE + c_destchoice_foreign * FOREIGN * ALT{0}_FOREIGN / ALT{0}_POP + c_destchoice_age_18_34 * ALT{0}_18_34 / ALT{0}_POP + c_destchoice_age_over_65 * ALT{0}_65 / ALT{0}_POP + c_destchoice_pctnobach * EDU_NOCOLLEGE * (ALT{0}_EDU_NOBACH + ALT{0}_EDU_NOHS + ALT{0}_EDU_HS) / ALT{0}_EMP + c_destchoice_pctbach * EDU_COLLEGE * ALT{0}_EDU_BACH / ALT{0}_EMP + c_destchoice_entscore * ALT{0}_ENT / AGEP / ALT{0}_EMP + c_destchoice_hhinc * ALT{0}_HHINC".format(num)
    exec(initialization)
print(V100)

# full model specification (takes a bit longer to run):
# "V{0} = log(ALT{0}_POP) + c_destchoice_dist * ALT{0}_DIST + c_destchoice_logdist * log(ALT{0}_DIST + 1) + c_destchoice_urban / DENS_ORIG * (ALT{0}_DENS - DENS_ORIG) + c_destchoice_hurent * ALT{0}_HURENT + c_destchoice_college * IN_COLLEGE * ALT{0}_COLLEGE + c_destchoice_vacancy * ALT{0}_VAC + c_destchoice_foreign * FOREIGN * ALT{0}_FOREIGN / ALT{0}_POP + c_destchoice_age_18_34 * ALT{0}_18_34 / ALT{0}_POP + c_destchoice_age_35_64 * ALT{0}_35_64 / ALT{0}_POP + c_destchoice_age_over_65 * ALT{0}_65 / ALT{0}_POP + c_destchoice_pctnobach * EDU_NOCOLLEGE * (ALT{0}_EDU_NOBACH + ALT{0}_EDU_NOHS + ALT{0}_EDU_HS) / ALT{0}_EMP + c_destchoice_pctbach * EDU_COLLEGE * ALT{0}_EDU_BACH / ALT{0}_EMP + c_destchoice_entscore * ALT{0}_ENT / AGEP / ALT{0}_EMP"

# "V{0} = log(ALT{0}_POP) + c_destchoice_dist * ALT{0}_DIST + c_destchoice_logdist * log(ALT{0}_DIST + 1) + c_adm * ADM * ALT{0}_ADM/ALT{0}_EMP + c_agr * AGR * ALT{0}_AGR/ALT{0}_EMP + c_con * CON * ALT{0}_CON/ALT{0}_EMP + c_edu * EDU * ALT{0}_EDU/ALT{0}_EMP + c_ent * ENT * ALT{0}_ENT/ALT{0}_EMP + c_ext * EXT * ALT{0}_EXT/ALT{0}_EMP + c_fin * FIN * ALT{0}_FIN/ALT{0}_EMP + c_fod * FOD * ALT{0}_FOD/ALT{0}_EMP + c_inf * INF * ALT{0}_INF/ALT{0}_EMP + c_med * MED * ALT{0}_MED/ALT{0}_EMP + c_mfg * MFG * ALT{0}_MFG/ALT{0}_EMP + c_mng * MNG * ALT{0}_MNG/ALT{0}_EMP + c_prf * PRF * ALT{0}_PRF/ALT{0}_EMP + c_pub * PUB * ALT{0}_PUB/ALT{0}_EMP + c_rel * REL * ALT{0}_REL/ALT{0}_EMP + c_ret * RET * ALT{0}_RET/ALT{0}_EMP + c_srv * SRV * ALT{0}_SRV/ALT{0}_EMP + c_trn * TRN * ALT{0}_TRN/ALT{0}_EMP + c_utl * UTL * ALT{0}_UTL/ALT{0}_EMP + c_whl * WHL * ALT{0}_WHL/ALT{0}_EMP"

# for the fields already in the Biogeme db.Database, can explicitly refer to them; also used a few references to other databases using .loc and fields in the Biogeme database

((((((((((((log(ALT100_POP) + (c_destchoice_dist(0) * ALT100_DIST)) + (c_destchoice_logdist(0) * log((ALT100_DIST + `1`)))) + ((c_destchoice_urban(0) / DENS_ORIG) * (ALT100_DENS - DENS_ORIG))) + (c_destchoice_hurent(0) * ALT100_HURENT)) + ((c_destchoice_college(0) * IN_COLLEGE) * ALT100_COLLEGE)) + (((c_destchoice_foreign(0) * FOREIGN) * ALT100_FOREIGN) / ALT100_POP)) + ((c_destchoice_age_18_34(0) * ALT100_18_34) / ALT100_POP)) + ((c_destchoice_age_over_65(0) * ALT100_65) / ALT100_POP)) + (((c_destchoice_pctnobach(0) * EDU_NOCOLLEGE) * ((ALT100_EDU_NOBACH + ALT100_EDU_NOHS) + ALT100_EDU_HS)) / ALT100_EMP)) + (((c_destchoice_pctbach(0) * EDU_COLLEGE) * ALT100_EDU_BACH) / ALT100_EMP)) + (((c_destchoice_entscore(0) * ALT100_ENT) / AGEP) / ALT100_EMP)) + (c_destchoice_hhinc(0) * ALT100_HHINC))


In [30]:
utilities = {}
for i in range(101):
    init = "utilities[{0}] = V{0}".format(i)
    exec(init)

In [31]:
import biogeme.messaging as msg
logger = msg.bioMessage()
logger.setDetailed()

In [32]:
logger.allMessages()

'*** Messages from biogeme 3.2.8 [2022-06-01]\n'

In [33]:
# Associate utility functions with the numbering of alternatives (corresponds to the CHOSEN field created earlier)
V = utilities

# Associate the availability conditions with the alternatives
# for this model, all migrants had all alternatives theoretically available so all are equal to 1 (available)
# if individual people had different availability for alterantives, could pass in a column of the dataframe to account for that availability
av = {}
for i in range(0, 101):
    av[i] = 1

# Definition of the model. This is the contribution of each
# observation to the log likelihood function.
# estimating the CHOSEN field
logprob = models.loglogit(V, av, CHOSEN)


# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'us_full_basic'

# Calculate the null log likelihood for reporting. (likelihood of predicting every entry's alterantive correctly if alternatives are randomly chosen)
biogeme.calculateNullLoglikelihood(av)

# Estimate the parameters
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

[10:06:33] < General >   Remove 2778 unused variables from the database as only 1521 are used.
[10:06:35] < Detailed >  It is suggested to scale the following variables.
[10:06:35] < Detailed >  Multiply ALT16_ENT by	1e-05 because the largest (abs) value is	80042.0
[10:06:35] < Detailed >  Multiply ALT8_DENS by	1e-05 because the largest (abs) value is	102846.33928097
[10:06:35] < Detailed >  Multiply ALT29_EDU_BACH by	1e-06 because the largest (abs) value is	371563.0
[10:06:35] < Detailed >  Multiply ALT98_EDU_HS by	1e-05 because the largest (abs) value is	150000.0
[10:06:35] < Detailed >  Multiply ALT10_POP by	1e-05 because the largest (abs) value is	301019.0
[10:06:35] < Detailed >  Multiply ALT14_EDU_NOHS by	1e-05 because the largest (abs) value is	96283.0
[10:06:35] < Detailed >  Multiply ALT54_DIST by	1e-07 because the largest (abs) value is	5587229.0
[10:06:35] < Detailed >  Multiply ALT57_DENS by	1e-05 because the largest (abs) value is	102846.33928097
[10:06:35] < Detailed >  M

In [34]:
df[["UNEMP_ORIG", "ORIGIN"]]

,UNEMP_ORIG,ORIGIN
0,0.048183,3400900
1,0.057408,5310200
2,0.068618,4801400
3,0.054169,4801200
4,0.063164,1207300
...,...,...
63267,0.056103,603700
63268,0.060474,607700
63269,0.053630,3704990
63270,0.039232,608500


In [35]:
migpuma_acs_data = pd.read_csv("data/ACS_MIGPUMA_2018.csv").set_index("MIGPUMA")

In [36]:
migpuma_acs_data.loc[3400900, ["Population 16 Years and Over in Labor Force Civilian Unemployed", "Population 16 Years and Over in Labor Force Civilian"]]

Population 16 Years and Over in Labor Force Civilian Unemployed     20995.0
Population 16 Years and Over in Labor Force Civilian               435731.0
Name: 3400900, dtype: float64